# SSD300 Pascal VOC Evaluation Tutorial

This is a brief tutorial that goes over how to evaluate the SSD300 '07+12+COCO' model (i.e. the version trained on MS COCO and afterwards fine-tuned on Pascal VOC) on the Pascal VOC2007 test dataset using the official Matlab evaluation script that comes with the [`VOCdevkit`](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html).

Of course the evaulation procedure described here is identical for SSD512, you just need to build a different model. Make sure that you build the model with the correct set of scaling factors to reproduce the official results. The models that were trained on MS COCO and fine-tuned on Pascal VOC need the MS COCO scaling factors.

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt

from keras_ssd300 import ssd_300
from keras_ssd_loss import SSDLoss
from keras_layer_AnchorBoxes import AnchorBoxes
from keras_layer_L2Normalization import L2Normalization
from ssd_box_encode_decode_utils import SSDBoxEncoder
from ssd_batch_generator import BatchGenerator
from pascal_voc_utils import predict_all_to_txt

%matplotlib inline

In [2]:
# Set the input image size for the model.
img_height = 300
img_width = 300

## 1. Load a trained SSD

Either load a trained model or build a model and load trained weights into it. Since the HDF5 files I'm providing contain only the weights for the various SSD versions, not the complete models, you'll have to go with the latter option when using this implementation for the first time. You can then of course save the model and next time load the full model directly, without having to build it.

You can find the download links to all the trained model weights in the README.

### 1.1. Build the model and load trained weights into it

In [ ]:
# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=20,
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05], # The scales for Pascal VOC are [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                limit_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                coords='centroids',
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=True)

# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
weights_path = 'path/to/trained/weights/VGG_VOC0712_SSD_300x300_ft_iter_120000.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=5e-04)

ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

Or

### 1.2. Load a trained model

In [ ]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = 'path/to/trained/model.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'compute_loss': ssd_loss.compute_loss})

## 2. Create a data generator for the evaluation dataset

Instantiate a `BatchGenerator` that will serve the evaluation dataset during the prediction phase.

In [14]:
dataset = BatchGenerator(box_output_format=['class_id', 'xmin', 'ymin', 'xmax', 'ymax'])

# TODO: Set the paths to the dataset here.
Pascal_VOC_dataset_images_dir = '../../datasets/VOCdevkit/VOC2007/JPEGImages/'
Pascal_VOC_dataset_annotations_dir = '../../datasets/VOCdevkit/VOC2007/Annotations/'
Pascal_VOC_dataset_image_set_filename = '../../datasets/VOCdevkit/VOC2007/ImageSets/Main/test.txt'

dataset.parse_xml(images_dirs=[Pascal_VOC_dataset_images_dir],
                  image_set_filenames=[Pascal_VOC_dataset_image_set_filename],
                  annotations_dirs=[Pascal_VOC_dataset_annotations_dir],
                  classes=classes,
                  include_classes='all',
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False)

## 3. Run the predictions over the evaluation dataset

Now that we have instantiated a model and a data generator to serve the dataset, we can make predictions on the entire dataset and save those predictions in text files in the appropriate format in which the Pascal VOC evaluation server (or the evaluation Matlab script) needs them for the evaluation.

Read the documenation to learn what the arguments mean, but the arguments as preset below reflect the parameters used in the evaluation of the original Caffe models.

In [6]:
# TODO: Set the batch size.
batch_size = 8

In [8]:
predict_all_to_txt(model=model,
                   img_height=img_height,
                   img_width=img_width,
                   batch_generator=dataset,
                   batch_size=batch_size,
                   batch_generator_mode='resize',
                   classes=['background',
                            'aeroplane', 'bicycle', 'bird', 'boat',
                            'bottle', 'bus', 'car', 'cat',
                            'chair', 'cow', 'diningtable', 'dog',
                            'horse', 'motorbike', 'person', 'pottedplant',
                            'sheep', 'sofa', 'train', 'tvmonitor'],
                   out_file_prefix='ssd300_07+12_2007_test_eval/comp3_det_test_',
                   confidence_thresh=0.01,
                   iou_threshold=0.45,
                   top_k=200,
                   pred_coords='centroids',
                   normalize_coords=True)

Number of images in the evaluation dataset: 4952
Producing results file: 100%|██████████| 619/619 [05:21<00:00,  2.11it/s]
All results files saved.


## 4. Run the evaluation

Now that we've generated the results text files for each class in the right format, we still have to evaluate the results.

This repository doesn't provide its own Pascal VOC mAP evaluation module at the moment, so you'll have to use the official Matlab evaluation script that comes with the `VOCdevkit`. Note that the mAP computation formula differs for VOC2007 and VOC2012, and the `VOCdevkit` for each of these years comes with its own evaluation script that you should use for the respective dataset.